# BookWise Distribution Center Optimization

## Problem Statement
BookWise Publishing is expanding their textbook distribution network in the Pacific Northwest. They need to optimize their warehouse locations to better serve their three main university bookstore partners. Management has approved opening 2-3 new distribution centers (DCs) from among 5 potential locations.

### Requirements:
- Must open at least 2 and at most 3 DCs
- Combined storage capacity must be at least 850,000 textbooks
- Each DC will make 40 shipments per year to each university bookstore

### Distribution Center Data:
| DC | Fixed Cost ($/year) | Transport Cost ($/km/shipment) | Capacity (1000 books) |
|----|--------------------|-------------------------------|---------------------|
| DC1| 85,000            | 0.55                          | 410                |
| DC2| 85,000            | 0.35                          | 380                |
| DC3| 75,000            | 0.65                          | 330                |
| DC4| 75,000            | 0.45                          | 380                |
| DC5| 85,000            | 0.25                          | 340                |

### Distance to University Bookstores (km):
| DC | Bookstore 1 | Bookstore 2 | Bookstore 3 |
|----|-------------|-------------|-------------|
| DC1| 70         | 110         | 65          |
| DC2| 50         | 135         | 80          |
| DC3| 60         | 115         | 85          |
| DC4| 55         | 90          | 95          |
| DC5| 95         | 65          | 85          |

---

In [3]:
# Required package installations (uncomment if needed)
# !pip install pulp
# !pip install numpy
# !pip install pandas

import pulp
import numpy as np
import pandas as pd

## Step 1: Set up the Linear Programming Problem

In [4]:
# Create the model
model = pulp.LpProblem("BookWise_DC_Optimization", pulp.LpMinimize)

# Decision variables (1 if DC is opened, 0 if not)
dc_vars = pulp.LpVariable.dicts("DC", range(1,6), cat='Binary')

# Data
fixed_costs = [85000, 85000, 75000, 75000, 85000]
capacities = [410, 380, 330, 380, 340]
transport_costs = [0.55, 0.35, 0.65, 0.45, 0.25]

distances = [
    [70, 110, 65],  # DC1
    [50, 135, 80],  # DC2
    [60, 115, 85],  # DC3
    [55, 90, 95],   # DC4
    [95, 65, 85]    # DC5
]

# Calculate total transport cost for each DC
transport_total = []
for i in range(5):
    dc_transport = sum(distances[i][j] * transport_costs[i] * 40 for j in range(3))
    transport_total.append(dc_transport)

# Objective function
model += pulp.lpSum((fixed_costs[i] + transport_total[i]) * dc_vars[i+1] for i in range(5))

# Constraints
# Minimum capacity
model += pulp.lpSum(capacities[i] * dc_vars[i+1] for i in range(5)) >= 850

# Number of DCs constraint
model += pulp.lpSum(dc_vars[i+1] for i in range(5)) >= 2
model += pulp.lpSum(dc_vars[i+1] for i in range(5)) <= 3

# Solve the model
model.solve()

print("Status:", pulp.LpStatus[model.status])

Status: Optimal


## Step 2: Analyze Results

In [5]:
# Print selected DCs
print("\nSelected Distribution Centers:")
selected_dcs = []
for i in range(5):
    if dc_vars[i+1].value() == 1:
        selected_dcs.append(i+1)
        print(f"DC{i+1}")

# Calculate total cost
total_cost = pulp.value(model.objective)
print(f"\nTotal Annual Cost: ${total_cost:,.2f}")

# Calculate capacity
total_capacity = sum(capacities[i] for i in range(5) if dc_vars[i+1].value() == 1)
print(f"Total Capacity: {total_capacity:,} thousand books")


Selected Distribution Centers:
DC3
DC4
DC5

Total Annual Cost: $248,530.00
Total Capacity: 1,050 thousand books


## Step 3: Cost Breakdown

In [6]:
print("Cost Breakdown for Selected DCs:")
for dc in selected_dcs:
    i = dc - 1
    fixed = fixed_costs[i]
    transport = transport_total[i]
    print(f"\nDC{dc}:")
    print(f"Fixed Cost: ${fixed:,.2f}")
    print(f"Transport Cost: ${transport:,.2f}")
    print(f"Total Cost: ${(fixed + transport):,.2f}")

Cost Breakdown for Selected DCs:

DC3:
Fixed Cost: $75,000.00
Transport Cost: $6,760.00
Total Cost: $81,760.00

DC4:
Fixed Cost: $75,000.00
Transport Cost: $4,320.00
Total Cost: $79,320.00

DC5:
Fixed Cost: $85,000.00
Transport Cost: $2,450.00
Total Cost: $87,450.00


## Conclusions

The optimization model has determined:

1. **Optimal DC Locations:** [Will be filled in after running]
2. **Total Annual Cost:** [Will be filled in after running]
3. **Total Capacity:** [Will be filled in after running]

This solution:
- Meets the minimum capacity requirement of 850,000 books
- Satisfies the constraint of opening 2-3 DCs
- Minimizes the combined fixed and transportation costs

The selected configuration provides the most cost-effective way to serve all three university bookstores while meeting all operational constraints.